#  Data Aggregation & Grouping  
Performing grouping, merging, and summarization with Pandas.


In [1]:
import pandas as pd

## Load the Dataset  
Load the Students Performance dataset from GitHub and display basic information and sample rows.


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/os7i/Data-Analysis-Preprocessing-/refs/heads/main/StudentsPerformance.csv')

print(" Data loaded successfully!")
df.info()
df.head()


 Data loaded successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB


,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


##  Merge Example Using Sample Data  
Split the dataset into training and testing parts, create two small DataFrames, and merge them based on the 'gender' column.


In [3]:
# Create two small DataFrames and merge them based on the 'gender' column

df1 = pd.DataFrame({"gender": ["male", "female"], "math score": [75, 88]})
df2 = pd.DataFrame({"gender": ["male", "female"], "reading score": [70, 90]})

df_merged = pd.merge(df1, df2, on="gender", how="inner")
print("Merged Data:")
df_merged


Merged Data:


,gender,math score,reading score
0,male,75,70
1,female,88,90


##  Create Students and Subjects DataFrames


In [4]:

students = pd.DataFrame({
    "StudentID": [1, 2, 3, 4],
    "StudentGender": df["gender"].head(4).values,
    "MathScore": df["math score"].head(4).values,
    "ReadingScore": df["reading score"].head(4).values,
    "WritingScore": df["writing score"].head(4).values
})

subjects = pd.DataFrame({
    "SubjectID": [101, 102, 103, 104],
    "SubjectName": ["Math", "Reading", "Writing", "Science"]
})

print("Students Dataset:")
display(students)

print("\nSubjects Dataset:")
display(subjects)


Students Dataset:


,StudentID,StudentGender,MathScore,ReadingScore,WritingScore
0,1,female,72,72,74
1,2,female,69,90,88
2,3,female,90,95,93
3,4,male,47,57,44



Subjects Dataset:


,SubjectID,SubjectName
0,101,Math
1,102,Reading
2,103,Writing
3,104,Science


##  Merge Operations (Inner & Left Join)


In [5]:

students_subjects = students.copy()
students_subjects["SubjectID"] = [101, 101, 102, 103]  # توزيع افتراضي للتجربة

inner_join = pd.merge(students_subjects, subjects, on="SubjectID", how="inner")
print("Inner Join Result:")
display(inner_join)

left_join = pd.merge(subjects, students_subjects, on="SubjectID", how="left")
print("Left Join Result:")
display(left_join)


Inner Join Result:


,StudentID,StudentGender,MathScore,ReadingScore,WritingScore,SubjectID,SubjectName
0,1,female,72,72,74,101,Math
1,2,female,69,90,88,101,Math
2,3,female,90,95,93,102,Reading
3,4,male,47,57,44,103,Writing


Left Join Result:


,SubjectID,SubjectName,StudentID,StudentGender,MathScore,ReadingScore,WritingScore
0,101,Math,1.0,female,72.0,72.0,74.0
1,101,Math,2.0,female,69.0,90.0,88.0
2,102,Reading,3.0,female,90.0,95.0,93.0
3,103,Writing,4.0,male,47.0,57.0,44.0
4,104,Science,NaN,NaN,NaN,NaN,NaN


##  Aggregations: Average, Count, and Range


In [6]:
# ============================================
# Aggregations: Average, Count, and Range (على عينة students)
# ============================================

avg_math_score = (
    students.groupby("StudentGender", as_index=False)["MathScore"]
    .mean()
    .rename(columns={"MathScore": "AverageMathScore"})
)

print("Average Math Score by Gender:")
display(avg_math_score)

student_count = (
    students.groupby("StudentGender", as_index=False)
    .size()
    .rename(columns={"size": "StudentCount"})
)

print("\nStudent Count by Gender:")
display(student_count)

score_range_data = (
    students.groupby("StudentGender", as_index=False)["MathScore"]
    .agg(ScoreRange=lambda x: x.max() - x.min())
)

print("\nScore Range by Gender:")
display(score_range_data)


Average Math Score by Gender:


,StudentGender,AverageMathScore
0,female,77.0
1,male,47.0



Student Count by Gender:


,StudentGender,StudentCount
0,female,3
1,male,1



Score Range by Gender:


,StudentGender,ScoreRange
0,female,21
1,male,0


##  Save Final Results


In [10]:
# Goal: Average score per subject by gender (full dataset)

# Steps:
# 1) Convert data to long format
# 2) Group by subject and gender
# 3) Merge with subjects to add SubjectID


# Convert to long format (Subject + Score)
long_df = df.melt(
    id_vars=["gender"],
    value_vars=["math score", "reading score", "writing score"],
    var_name="SubjectName",
    value_name="Score"
)

# Clean subject names
long_df["SubjectName"] = (
    long_df["SubjectName"]
    .str.replace(" score", "", regex=False)
    .str.title()
)

# Aggregate metrics
grouped = (
    long_df.groupby(["SubjectName", "gender"], as_index=False)
    .agg(
        AverageScore=("Score", "mean"),
        StudentCount=("Score", "size"),
        MinScore=("Score", "min"),
        MaxScore=("Score", "max")
    )
    .rename(columns={"gender": "StudentGender"})
)

# Merge with subjects to add SubjectID
final_data = (
    grouped.merge(subjects, on="SubjectName", how="left")[
        ["SubjectID", "SubjectName", "StudentGender",
         "AverageScore", "StudentCount", "MinScore", "MaxScore"]
    ]
    .sort_values(["SubjectID", "StudentGender"], na_position="last")
)

print("Final Grouped Data (Subject x Gender):")
display(final_data)


Final Grouped Data (Subject x Gender):


,SubjectID,SubjectName,StudentGender,AverageScore,StudentCount,MinScore,MaxScore
0,101,Math,female,63.633205,518,0,100
1,101,Math,male,68.728216,482,27,100
2,102,Reading,female,72.608108,518,17,100
3,102,Reading,male,65.473029,482,23,100
4,103,Writing,female,72.467181,518,10,100
5,103,Writing,male,63.311203,482,15,100


##  Save the Final Grouped Data


In [11]:
# Save the grouped and summarized dataset
final_data.to_csv('Students_Grouped_Data.csv', index=False)
print("File saved successfully as 'Students_Grouped_Data.csv'")


File saved successfully as 'Students_Grouped_Data.csv'
